In [48]:
#LOAD DATA
from dataclasses import dataclass
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display

from Quantapp.Universe import Universe
plt.rcParams["figure.figsize"] = (20, 7)

import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from numpy_ext import rolling_apply

from Quantapp.Algorithm   import Algorithm
from Quantapp.Computation import Computation
from Quantapp.Portfolio   import Portfolio
from Quantapp.Plot        import Plot
from Quantapp.DataManager import DataManager
from Quantapp.Universe    import Universe

csv_file_paths = [
    'csv_files/Broad Market.csv',
    'csv_files/S&P 500.csv',
    'csv_files/Russell 1000.csv'
]

dm   = DataManager()
universe  = Universe(file_paths = csv_file_paths)
pm   = Portfolio(universe = universe)
comp = Computation()
plot = Plot()
algorithm = Algorithm()

tickers = pm.add_csv('csv_files/portfolio')


pm.load(period='3y',interval='1d');


benchmark      = dm.retrieve_ticker_data('SPY', period='3y', interval='1d')
risk_free_rate = dm.retrieve_ticker_data('^IRX', period='3y',interval='1d').reindex_like(benchmark)
scalar_multiplier = .7




In [ ]:
#percentage of each asset group

values = pm.retrieve_sectors()
names = list(values.keys())
values = list(values.values())

sector_exposure = px.pie(values=values, names=names,title='Diversity Exposure by Sector')
sector_exposure.show()

details = pm.position_details.copy()
details['BP Usage %'] = details['BP Usage %'].apply(lambda s: float(s.replace('%', '')))
details = details.groupby('Sector').sum()
values  = list(details['BP Usage %'])
names   = list(details.index)

buying_power_exposure = px.pie(values=values,names=names,title='Buying Exposure Percentage by Sector')
buying_power_exposure.show()


In [52]:
#Average Portfolio Performance 
'''' Multiple windows (50 , 200)
1. Average Relative Momentum 
2. Average Portfolio Sharpe
3. Average Portfolio Alpha
4. Average Beta
5. Average Portfolio Volatility (Downside Standard Deviation, Standard Deviation, Yang Zhang)
'''
'''
mtf_average_roc_portfolio = pd.DataFrame()
portfolio_roc_50 = pm.rolling(algorithm.rate_of_change, 50)
portfolio_roc_200 = pm.rolling(algorithm.rate_of_change, 200)
average_portfolio_roc_50  = portfolio_roc_50.mean(axis=1)
average_portfolio_roc_200 = portfolio_roc_200.mean(axis=1)
mtf_average_roc_portfolio['roc_50']  = average_portfolio_roc_50
mtf_average_roc_portfolio['roc_200'] = average_portfolio_roc_200

mtf_average_sharpe_portfolio = pd.DataFrame()
portfolio_sharpe_50 = pm.rolling(algorithm.sharpe, 50, risk_free_rate['Close'])
portfolio_sharpe_200 = pm.rolling(algorithm.sharpe, 200,risk_free_rate['Close'])
average_portfolio_sharpe_50  = portfolio_sharpe_50.mean(axis=1)
average_portfolio_sharpe_200 = portfolio_sharpe_200.mean(axis=1)
mtf_average_sharpe_portfolio['sharpe_50']  = average_portfolio_sharpe_50
mtf_average_sharpe_portfolio['sharpe_200'] = average_portfolio_sharpe_200

mtf_average_alpha_portfolio = pd.DataFrame()
portfolio_alpha_50 = pm.rolling(algorithm.alpha, 50,benchmark['Close'])
portfolio_alpha_200 = pm.rolling(algorithm.alpha, 200,benchmark['Close'])
average_portfolio_alpha_50  = portfolio_alpha_50.mean(axis=1)
average_portfolio_alpha_200 = portfolio_alpha_200.mean(axis=1)
mtf_average_alpha_portfolio['alpha_50']  = average_portfolio_alpha_50
mtf_average_alpha_portfolio['alpha_200'] = average_portfolio_alpha_200

mtf_average_beta_portfolio = pd.DataFrame()
portfolio_beta_50  = pm.rolling(algorithm.beta, 50,  benchmark['Close'])
portfolio_beta_200 = pm.rolling(algorithm.beta, 200, benchmark['Close'])
average_portfolio_beta_50  = portfolio_beta_50.mean(axis=1)
average_portfolio_beta_200 = portfolio_beta_200.mean(axis=1)
mtf_average_beta_portfolio['beta_50']  = average_portfolio_beta_50
mtf_average_beta_portfolio['beta_200'] = average_portfolio_beta_200

mtf_average_std_portfolio = pd.DataFrame()
portfolio_std_50  = pm.rolling(algorithm.standard_deviation, 50)
portfolio_std_200 = pm.rolling(algorithm.standard_deviation, 200)
average_portfolio_std_50  = portfolio_std_50.mean(axis=1)
average_portfolio_std_200 = portfolio_std_200.mean(axis=1)
mtf_average_std_portfolio['std_50']  = average_portfolio_std_50
mtf_average_std_portfolio['std_200'] = average_portfolio_std_200

mtf_average_semi_std_portfolio = pd.DataFrame()
portfolio_semi_std_50  = pm.rolling(algorithm.semi_standard_deviation, 50)
portfolio_semi_std_200 = pm.rolling(algorithm.semi_standard_deviation, 200)
average_portfolio_semi_std_50       = portfolio_semi_std_50.mean(axis=1)
average_portfolio_semi_std_200      = portfolio_semi_std_200.mean(axis=1)
mtf_average_semi_std_portfolio['semi_std_50']  = average_portfolio_semi_std_50
mtf_average_semi_std_portfolio['semi_std_200'] = average_portfolio_semi_std_200

benchmark_roc_200      = comp.rolling(algorithm.rate_of_change,200,benchmark['Close'])
benchmark_sharpe_200   = comp.rolling(algorithm.sharpe,200,benchmark['Close'],risk_free_rate['Close'])
benchmark_std_200      = comp.rolling(algorithm.standard_deviation,200,benchmark['Close'])
benchmark_semi_std_200 = comp.rolling(algorithm.semi_standard_deviation,200,benchmark['Close'])

fig_1 = px.line(portfolio_roc_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_2 = px.line(portfolio_sharpe_200.dropna() ,height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_3 = px.line(portfolio_alpha_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_4 = px.line(portfolio_beta_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_5 = px.line(portfolio_std_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_6 = px.line(portfolio_semi_std_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)

fig_1_average = px.line(mtf_average_roc_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_2_average = px.line(mtf_average_sharpe_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_3_average = px.line(mtf_average_alpha_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_4_average = px.line(mtf_average_beta_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_5_average = px.line(mtf_average_std_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_6_average = px.line(mtf_average_semi_std_portfolio.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)


fig_1_benchmark = px.line(benchmark_roc_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_2_benchmark = px.line(benchmark_sharpe_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_5_benchmark = px.line(benchmark_std_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
fig_6_benchmark = px.line(benchmark_semi_std_200.dropna(),height=800 * scalar_multiplier,width=2200 * scalar_multiplier)

title_tuple=("Portfolio: Rate of Change", "Individual Assets: Rate of Change",
             "Portfolio: Sharpe"        , "Invididual Assets: Sharpe",
             "Portfolio: Alpha"        , "Invididual Assets: Alpha",
             "Portfolio: Beta"        , "Invididual Assets: Beta",
             "Portfolio: Standard Deviation"        , "Invididual Assets: Standard Deviation")

_fig_layout = make_subplots(rows=6, cols=2, subplot_titles=title_tuple)
scalar_multiplier = .9
_fig_layout['layout'].update(height=1800* scalar_multiplier * 2, width=1000 * scalar_multiplier*2)

for n in range(len(tickers)):
    _fig_layout.add_trace(fig_1.data[n], col=2,row=1)
    _fig_layout.add_trace(fig_2.data[n], col=2,row=2)
    _fig_layout.add_trace(fig_3.data[n], col=2,row=3)
    _fig_layout.add_trace(fig_4.data[n], col=2,row=4)
    _fig_layout.add_trace(fig_5.data[n], col=2,row=5)
    _fig_layout.add_trace(fig_6.data[n], col=2,row=6)

for n in range(2):
    _fig_layout.add_trace(fig_1_average.data[n], col=1,row=1)
    _fig_layout.add_trace(fig_2_average.data[n], col=1,row=2)
    _fig_layout.add_trace(fig_3_average.data[n], col=1,row=3)
    _fig_layout.add_trace(fig_4_average.data[n], col=1,row=4)
    _fig_layout.add_trace(fig_5_average.data[n], col=1,row=5)
    _fig_layout.add_trace(fig_6_average.data[n], col=1,row=6)

for n in range(2):
    _fig_layout.add_trace(fig_1_average.data[n], col=1,row=1)
    _fig_layout.add_trace(fig_2_average.data[n], col=1,row=2)
    _fig_layout.add_trace(fig_3_average.data[n], col=1,row=3)
    _fig_layout.add_trace(fig_4_average.data[n], col=1,row=4)
    _fig_layout.add_trace(fig_5_average.data[n], col=1,row=5)
    _fig_layout.add_trace(fig_6_average.data[n], col=1,row=6)

#_fig_layout.add_trace(fig_1_benchmark.data[0], col=1, row=1)
#_fig_layout.add_trace(fig_2_benchmark.data[0], col=1, row=2)
#_fig_layout.add_trace(fig_5_benchmark.data[0], col=1, row=5)
#_fig_layout.add_trace(fig_6_benchmark.data[0], col=1, row=6)

_fig_layout.add_hline(y=0, col=2,row=1, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=0, col=2,row=2, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=0, col=2,row=3, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=1, col=2,row=4, line_width=2, line_dash="dash", line_color="grey")

_fig_layout.add_hline(y=0, col=1,row=1, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=0, col=1,row=2, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=0, col=1,row=3, line_width=2, line_dash="dash", line_color="grey")
_fig_layout.add_hline(y=1, col=1,row=4, line_width=2, line_dash="dash", line_color="grey")

_fig_layout
'''
dontrun = 1



In [51]:
#Portfolio Constituent Performance rankings

beta_calculations_21  = pm.rolling(algorithm.beta,21,benchmark['Close'])
sharpe_calculations_21 = pm.rolling(algorithm.sharpe,21,risk_free_rate['Close'])
alpha_calculations_21  = pm.rolling(algorithm.alpha, 21,benchmark['Close'])

beta_calculations_50   = pm.rolling(algorithm.beta,50,benchmark['Close'])
sharpe_calculations_50 = pm.rolling(algorithm.sharpe,50,risk_free_rate['Close'])
alpha_calculations_50  = pm.rolling(algorithm.alpha, 50,benchmark['Close'])

beta_calculations_200   = pm.rolling(algorithm.beta,200,benchmark['Close'])
sharpe_calculations_200 = pm.rolling(algorithm.sharpe,200,risk_free_rate['Close'])
alpha_calculations_200   = pm.rolling(algorithm.alpha, 200,benchmark['Close'])


beta_rank_21   = beta_calculations_21.abs().rank(method='dense',axis=1,ascending=True).fillna(0.0).astype(int)
sharpe_rank_21 = sharpe_calculations_21.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)
alpha_rank_21  = alpha_calculations_21.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)

beta_rank_50   = beta_calculations_50.abs().rank(method='dense',axis=1,ascending=True).fillna(0.0).astype(int)
sharpe_rank_50 = sharpe_calculations_50.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)
alpha_rank_50  = alpha_calculations_50.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)

beta_rank_200   = beta_calculations_200.abs().rank(method='dense',axis=1,ascending=True).fillna(0.0).astype(int)
sharpe_rank_200 = sharpe_calculations_200.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)
alpha_rank_200  = alpha_calculations_200.rank(method='dense',axis=1,ascending=False).fillna(0.0).astype(int)


ranks_21 = (beta_rank_21 + sharpe_rank_21 + alpha_rank_21).rank(method='dense',axis=1,ascending=True).astype(int)[-30:]
ranks_50 = (beta_rank_50 + sharpe_rank_50 + alpha_rank_50).rank(method='dense',axis=1,ascending=True).astype(int)[-30:]
ranks_200 = (beta_rank_200+ sharpe_rank_200+ alpha_rank_200).rank(method='dense',axis=1,ascending=True).astype(int)[-30:]

top_num = 10


last_row_21D = ranks_50.iloc[-1]
top_21D    = list(last_row_21D [last_row_21D  < (top_num + 1)].index)
bottom_21D = list(last_row_21D [last_row_21D  > (pd.Series(last_row_21D .unique()).max()-top_num)].index)

last_row_50D = ranks_50.iloc[-1]
top_50D    = list(last_row_50D [last_row_50D  < (top_num + 1)].index)
bottom_50D = list(last_row_50D [last_row_50D  > (pd.Series(last_row_50D .unique()).max()-top_num)].index)

last_row_200D = ranks_200.iloc[-1]
top_200D    = list(last_row_200D [last_row_200D  < (top_num + 1)].index)
bottom_200D = list(last_row_200D [last_row_200D  > (pd.Series(last_row_200D .unique()).max()-top_num)].index)

top_performance_chart_21 = px.line(ranks_21[top_21D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='TOP 21 Day Portfolio Performance Rankings')
bottom_performance_chart_21 = px.line(ranks_21[bottom_21D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='BOTTOM 21 Day Portfolio Performance Rankings')

top_performance_chart_50 = px.line(ranks_50[top_50D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='TOP 50 Day Portfolio Performance Rankings')
bottom_performance_chart_50 = px.line(ranks_50[bottom_50D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='BOTTOM 50 Day Portfolio Performance Rankings')

top_performance_chart_200= px.line(ranks_200[top_200D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='TOP 200 Day Portfolio Performance Rankings')
bottom_performance_chart_200= px.line(ranks_200[bottom_200D],height=600*scalar_multiplier,width=1600*scalar_multiplier,title='BOTTOM 200 Day Portfolio Performance Rankings')

top_performance_chart_21.show()
bottom_performance_chart_21.show()

top_performance_chart_50.show()
bottom_performance_chart_50.show()

top_performance_chart_200.show()
bottom_performance_chart_200.show()

C:\Users\kevin\AppData\Roaming\Python\Python39\site-packages\pandas\core\nanops.py:1582: RuntimeWarning:

Degrees of freedom <= 0 for slice

C:\Users\kevin\AppData\Roaming\Python\Python39\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning:

divide by zero encountered in true_divide



In [54]:
#Average Rolling correlation of portfolio with Broader Markets

broad_markets = {
    'SPY' : 'long'
}


time_frames = [21,50,200]

broad_market_portfolio = Portfolio(universe=universe)
broad_market_portfolio.add_tickers(broad_markets)

pm.load(period='3y', interval='1d')
broad_market_portfolio.load(period='3y', interval='1d')

broad_market_data = broad_market_portfolio.retrieve_ticker_data()
portfolio_data    = pm.retrieve_ticker_data()

stock_market_corr = pm.rolling_multi_timeframe(algorithm.correlation, time_frames, broad_market_data['SPY'])

broad_market_portfolio.retrieve_assets().keys

def average_portfolio_correlation(market_df):
    average_window_df = pd.DataFrame()
    for window in time_frames:
        df = pd.DataFrame()
        for key in pm.retrieve_assets().keys():
            df[str(key) + '_' + str(window)]=market_df[key][window]
        average_window_df['Average_' + str(window)] = df.mean(axis=1)
    return average_window_df

stock_market_corr_chart         = px.line(average_portfolio_correlation(stock_market_corr),title='Stock market Correlation',height=800 * scalar_multiplier,width=2200 * scalar_multiplier)
stock_market_corr_chart.add_hline(y=0, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1).show()


C:\Users\kevin\AppData\Roaming\Python\Python39\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning:

Degrees of freedom <= 0 for slice

c:\Users\kevin\Dropbox\Code\Notebooks\Projects\Quantapp\Computation.py:49: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [ ]:
#Average internal correlation of each individual assets with other portfolio components

pm.load(period='1y')
twenty_d= pm.retrieve_ticker_data()[-21:]

pm.load(period='1y')
fifty_d= pm.retrieve_ticker_data()[-50:]

pm.load(period='1y')
two_hundred_d = pm.retrieve_ticker_data()[-200:]

corr_21D_matrix   = comp.create_correlation_heatmap(twenty_d)
corr_50D_matrix   = comp.create_correlation_heatmap(fifty_d)
corr_200D_matrix  = comp.create_correlation_heatmap(two_hundred_d)

scalar_multiplier = .7

twenty_d_corr      = twenty_d .corr().round(2).mean(axis=0).sort_values(ascending=False)
fifty_d_corr       = fifty_d .corr().round(2).mean(axis=0).sort_values(ascending=False)
two_hundred_d_corr = two_hundred_d.corr().round(2).mean(axis=0).sort_values(ascending=False)

s1, s2 = fifty_d_corr.align(two_hundred_d_corr)
_, s3 = twenty_d_corr.align(fifty_d_corr)

difference_corr = ((s1 + s2 + s3) / 3).sort_values(ascending=False)

#average_corr_2 = px.bar(fifty_d_corr, height=800 * scalar_multiplier,width=2400* scalar_multiplier,title='50 Day average internal correlation')
#average_corr_3 = px.bar(two_hundred_d_corr, height=800 * scalar_multiplier,width=2400* scalar_multiplier,title='200 Day average internal correlation')
average_corr_4 = px.bar(difference_corr, height=800 * scalar_multiplier,width=2400* scalar_multiplier,title='Average of 21,50,200 Day internal correlation')

#average_corr_2.add_hline(y=0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
#average_corr_3.add_hline(y=0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)
average_corr_4.add_hline(y=0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1)

#average_corr_2.add_hline(y=-0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1).show()
#average_corr_3.add_hline(y=-0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1).show()
average_corr_4.add_hline(y=-0.25, line_dash="dash", row='all', col='all', line_color="#000000", line_width=1).show()





In [ ]:
#Portfolio correlation heat map on multiple time frames 

fig_layout = make_subplots(rows=1,
                    cols=3,
                    subplot_titles=("21 Day Correlation","50 Day Correlation", "200 Day Correlation"),
                    vertical_spacing = 0.05)

fig_layout.update_layout(title_text="Portfolio Correlation")

scalar_multiplier = .7
fig_layout['layout'].update(height=900* scalar_multiplier, width=2400 * scalar_multiplier, title='')

ticker_data = pm.retrieve_ticker_data()

returns_21 = ticker_data[-21:]
returns_50 = ticker_data[-50:]
returns_200=ticker_data[-200:]


returns_21_fig  = plot.create_correlation_figure(returns_21)
returns_50_fig  = plot.create_correlation_figure(returns_50)
returns_200_fig = plot.create_correlation_figure(returns_200)

fig_layout.add_trace(returns_21_fig.data[0], col=1,row=1)
fig_layout.add_trace(returns_50_fig.data[0], col=2,row=1)
fig_layout.add_trace(returns_200_fig.data[0], col=3,row=1)
fig_layout.update_layout(title_text="Portfolio Correlation")




In [ ]:
#Multi-time frame 'Rank' of Highest, Lowest, and Least correlated Assets to portfolio

''' 
1. Retrieve all assets in portfolio 
2. Retrieve all assets in universe to compare
3. Calculate average correlation of each portfolio asset to each asset in universe 21,50,200 window
4. Rank each asset in universe by level of correlation by Highest, Lowest, least for each window
5. find the average rank for all windows for each asset and sort by average rank
6. display sorted table displaying each windows rank and average rank
'''

pm.load(period='3y',interval='1d')
portfolio_assets = pm.retrieve_ticker_data()



In [ ]:
#Options Portfolio

import yfinance as yf
msft = yf.Ticker("MSFT")
msft.options

for expiration in msft.options:
    print(msft.option_chain(expiration).calls)

# The usual py_vollib syntax

import numpy as np
import pandas as pd
import yfinance as yf
from Quantapp.Algorithms import Algorithms
data = yf.Ticker('SPY')
algorithms = Algorithms()

from py_vollib.black_scholes.greeks.analytical import *
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
from py_vollib.black_scholes import *



option_chain = data.option_chain()

strike_list = option_chain.calls['strike']
current_price = data.history(period='5y', interval='1D').iloc[-1]['Close']
volatility = algorithms.rolling(algorithms.standard_deviation, 200, data.history(period='1Y', interval='1D')['Close'])
sigma= volatility.iloc[-1] 
sigma /=10
sigma /=10
flag = 'c'  # 'c' for call, 'p' for put
F = 100  # Underlying asset price
K = 90  # Strike
t = 0.5  # (Annualized) time-to-expiration
r = 0.01  # Interest free rate

delta_list = []
gamma_list = []
theta_list = []
vega_list  = []
rho_list  = []
implied_volatility_list = []
price_list = []

for strike in strike_list:
    delta_list.append(delta(flag, current_price, strike, t, r, sigma))
    gamma_list.append(gamma(flag, current_price, strike, t, r, sigma))
    theta_list.append(theta(flag, current_price, strike, t, r, sigma))
    vega_list.append(vega(flag, current_price, strike, t, r, sigma))
    rho_list.append(rho(flag, current_price, strike, t, r, sigma))
    price = black_scholes(flag, current_price, strike, t, r, sigma)
    price_list.append(price)
    implied_volatility_list.append(iv(price, current_price, strike, t, r, flag))
  
greeks = {
    'strike': strike_list,
    'delta': delta_list,
    'gamma': gamma_list,
    'theta': theta_list,
    'vega' : vega_list,
    'rho': rho_list,
    'IV': implied_volatility_list,
    'price': price_list,
}


df = pd.DataFrame(greeks)
df.index = strike_list

df
#gamma(flag, F, K, t, r, sigma)